[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/giswqs/gee-tutorials/blob/master/AssetManagement/export_data.ipynb)

In [ ]:
# !pip install geemap

In [ ]:
import os
import ee
import geemap

In [ ]:
from geemap.datasets import DATA

In [ ]:
Map = geemap.Map()

## Set an output directory

In [ ]:
out_dir = os.path.expanduser("~/Downloads")

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

## Export an ee.FeatureCollection

In [ ]:
fc = ee.FeatureCollection('users/giswqs/public/countries')

In [ ]:
Map.addLayer(fc, {}, "Countries")
Map.centerObject(fc)
Map

Export data to a local computer

In [ ]:
out_shp = os.path.join(out_dir, "countries.shp")
geemap.ee_export_vector(fc, out_shp, verbose=True)

Export data to Google Drive

In [ ]:
geemap.ee_export_vector_to_drive(fc, description="countries", folder="export")

## Export an ee.Image

In [ ]:
image = ee.Image('LE7_TOA_5YEAR/1999_2003')

landsat_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'gamma': 1.7
}
Map.addLayer(image, landsat_vis, "LE7_TOA_5YEAR/1999_2003", True, 1)
Map

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'landsat.tif')

### Exporting all bands as one single image

In [ ]:
# Draw any shapes on the map using the Drawing tools before executing this code block
roi = Map.user_roi

In [ ]:
roi.getInfo()

In [ ]:
image_clip = image.clip(roi)

In [ ]:
Map.addLayer(image_clip, landsat_vis, "image")

In [ ]:
geemap.ee_export_image(image_clip, filename=filename, scale=90, region=roi, file_per_band=False)

### Exporting each band as one image

In [ ]:
geemap.ee_export_image(image, filename=filename, scale=90, region=roi, file_per_band=True)

### Export an image to Google Drive

In [ ]:
geemap.ee_export_image_to_drive(image, description='landsat', folder='export', region=roi, scale=30)

## Download an ee.ImageCollection

In [ ]:
loc = ee.Geometry.Point(-99.2222, 46.7816)
collection = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(loc) \
    .filterDate('2008-01-01', '2020-01-01') \
    .filter(ee.Filter.listContains("system:band_names", "N"))

In [ ]:
out_dir = os.path.expanduser('~/Downloads')

In [ ]:
collection.aggregate_array('system:index').getInfo()

In [ ]:
geemap.ee_export_image_collection(collection, out_dir=out_dir)

In [ ]:
geemap.ee_export_image_collection_to_drive(collection, folder='export', scale=10)

## Extract pixels as a Numpy array

In [ ]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt

img = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_038029_20180810') \
  .select(['B4', 'B5', 'B6'])

aoi = ee.Geometry.Polygon(
  [[[-110.8, 44.7],
    [-110.8, 44.6],
    [-110.6, 44.6],
    [-110.6, 44.7]]], None, False)

rgb_img = geemap.ee_to_numpy(img, region=aoi)
print(rgb_img.shape)

In [ ]:
# Scale the data to [0, 255] to show as an RGB image.
# Adapted from https://bit.ly/2XlmQY8. Credits to Justin Braaten
rgb_img_test = (255*((rgb_img[:, :, 0:3] - 100)/3500)).astype('uint8')
plt.imshow(rgb_img_test)
plt.show()